In [1]:
spark.sql("CREATE SCHEMA IF NOT EXISTS silver")

StatementMeta(, 0171335a-d725-4181-8308-a46732c2db5f, 3, Finished, Available, Finished)

DataFrame[]

In [1]:
df_bronze = spark.sql("SELECT * FROM bronze.nyc_taxi_trips_bronze")
df_bronze.printSchema()
df_bronze.show(5)


StatementMeta(, 51617e8e-becc-4087-93da-79b632dad1d6, 3, Finished, Available, Finished)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)
 |-- processed_timestamp: timestamp (nullable = true)
 |-- source_file: stri

In [3]:
from pyspark.sql.functions import col

df_silver = (
    df_bronze
    .filter(col("tpep_pickup_datetime").isNotNull())
    .filter(col("tpep_dropoff_datetime").isNotNull())
    .filter(col("trip_distance") > 0)
    .filter(col("fare_amount") > 0)
)


StatementMeta(, 51617e8e-becc-4087-93da-79b632dad1d6, 5, Finished, Available, Finished)

In [4]:
lookup_path = "Files/raw-landing/taxi_zone_lookup.csv"

df_zones = (
    spark.read
    .option("header", True)
    .csv(lookup_path)
)

df_zones.show(5)


StatementMeta(, 0171335a-d725-4181-8308-a46732c2db5f, 6, Finished, Available, Finished)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: abfss://9c8d15ad-a9b9-4d06-92eb-31efa9d6afee@onelake.dfs.fabric.microsoft.com/ed121242-37a4-45e7-8b9f-67493c433551/Files/raw-landing/taxi_zone_lookup.csv.

In [4]:
lookup_path = "Files/raw-landing/taxi_zone_lookup.csv"

df_zones = (
    spark.read
    .option("header", True)
    .csv(lookup_path)
)

df_zones.printSchema()
df_zones.show(5)


StatementMeta(, 51617e8e-becc-4087-93da-79b632dad1d6, 6, Finished, Available, Finished)

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [5]:
df_silver_enriched = (
    df_silver
    .join(
        df_zones,
        df_silver.PULocationID == df_zones.LocationID,
        "left"
    )
)

df_silver_enriched.show(5)


StatementMeta(, 51617e8e-becc-4087-93da-79b632dad1d6, 7, Finished, Available, Finished)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------------+--------------------+----------+---------+--------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee| processed_timestamp|         source_file|LocationID|  Borough|                Zone|service_zone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+--

In [6]:
(
    df_silver_enriched.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver.nyc_taxi_trips_silver")
)


StatementMeta(, 51617e8e-becc-4087-93da-79b632dad1d6, 8, Finished, Available, Finished)

In [7]:
spark.sql("""
SELECT
  COUNT(*) AS total_records,
  COUNT(DISTINCT Borough) AS borough_count
FROM silver.nyc_taxi_trips_silver
""").show()


StatementMeta(, 51617e8e-becc-4087-93da-79b632dad1d6, 9, Finished, Available, Finished)

+-------------+-------------+
|total_records|borough_count|
+-------------+-------------+
|     52066760|            8|
+-------------+-------------+

